In [ ]:
from docksuitex import Protein, Ligand, PocketFinder, VinaDocking, AD4Docking
from docksuitex.utils import clean_temp_folder, fetch_pdb, fetch_sdf, view_molecule, parse_vina_log_to_csv, parse_ad4_dlg_to_csv
clean_temp_folder()

In [ ]:
# Fetch and prepare protein
Protein_path = fetch_pdb(pdbid = "1HVR")
prot = Protein(Protein_path)
prot.prepare(fix_pdb =True, remove_heterogens=True, add_hydrogens=True, remove_water=True, add_charges=True, preserve_charge_types=None) #Default parameters
prot.save_pdbqt()
view_molecule(file_path = prot.pdbqt_path, style = "cartoon", background="white", color="spectrum", width=500, height=500)

In [ ]:
# Find pockets using p2rank
pf = PocketFinder(prot.pdb_path)
pockets = pf.run()
pf.save_report()
pockets

In [ ]:
# Fetch and prepare ligand
Ligand_path = fetch_sdf(cid = "338")
lig = Ligand(Ligand_path)
lig.prepare(minimize=None,  remove_water=True, add_hydrogens=True, add_charges=True, preserve_charge_types=None) #Default parameters
lig.save_pdbqt()
view_molecule(file_path = lig.pdbqt_path, style = "stick")

#### Perform protein ligand docking at the first pocket

In [ ]:
# Perform docking with the first pocket
# Using AutoDock4
ad4 = AD4Docking(
    receptor=prot.pdbqt_path,
    ligand=lig.pdbqt_path,
    grid_center=pockets[0]['center'],

    #Default Parameters
    grid_size=(60,60,60),
    spacing=0.375,     
    dielectric=-0.1465, 
    smooth=0.5,          
    ga_pop_size=150,
    ga_num_evals=2500000,
    ga_num_generations=27000,
    ga_elitism=1,
    ga_mutation_rate=0.02,
    ga_crossover_rate=0.8,
    ga_run=10,
    rmstol=2.0,
)

ad4.run()
ad4.save_results(f"ad4_results/{prot.pdbqt_path.stem}_{lig.pdbqt_path.stem}_pocket1_docking")

# Parse AD4 results
parse_ad4_dlg_to_csv("ad4_results", "ad4_results/ad4_summary.csv")

#### Perform protein ligand docking at all the predicted protein pockets


In [ ]:
# Loop through each pocket and perform docking
# Using AutoDock Vina
for pocket in pockets: #pockets[:3], For first three pockets
    center = pocket["center"]
    rank = pocket["rank"]

    vina = VinaDocking(
        receptor=prot.pdbqt_path,
        ligand=lig.pdbqt_path,
        grid_center=center,
        
        # Default Parameters
        grid_size=(60, 60, 60),
        exhaustiveness=8,
        num_modes=9
    )
    vina.run()
    vina.save_results(f"vina_results/{prot.pdbqt_path.stem}_{lig.pdbqt_path.stem}_pocket_{rank}_docking")


# Parse Vina results
parse_vina_log_to_csv("vina_results", "vina_results/vina_summary.csv")
